In [ ]:
import matplotlib.pyplot as plt
import os
import pandas as pd

workdir = "/work/data/apaatero/output/20200309"#/testD5_HJ"
#filename = "MyExpt_CellsN10.csv"
filename = "test_D5CellsN10.csv"
basename, extension = os.path.splitext(filename)

df = pd.read_csv(os.path.join(workdir, filename))
#df.head()

In [ ]:
df.drop(['Metadata_ChannelNumber', 'Metadata_FileLocation', 'Metadata_Frame', 'Metadata_Site', 'Metadata_Series'], axis=1, inplace=True)

# columns to group the results by
group_cols = ['Metadata_Plate', 'Metadata_Well', 'ImageNumber']

# https://stackoverflow.com/questions/45752601/how-to-do-a-conditional-count-after-groupby-on-a-pandas-dataframe
#df11=df.groupby(group_cols)['Children_Spots_Count'].apply(lambda x: (x>0).sum()).reset_index(name='cells_with_spots')
#df12=df.groupby(group_cols)['Children_Spots_Count'].apply(lambda x: (x==0).sum()).reset_index(name='cells_with_no_spots')

aggs = {'ObjectNumber': 'count', \
        'Classify_nospots': 'sum', \
        'Classify_spots': 'sum'}

# group cells by plate and well, aggregate by counting rows
totals_by_image = df.groupby(group_cols).agg(aggs).reset_index()

# rename object number columns
totals_by_image.rename(columns={"ObjectNumber": "Total cells"}, inplace=True)
totals_by_image.rename(columns={"Classify_nospots": "Cells with spots"}, inplace=True)
totals_by_image.rename(columns={"Classify_spots": "Spotted cells"}, inplace=True)

totals_by_image['Spotted fraction'] = totals_by_image['Spotted cells'] / totals_by_image['Total cells']
print(totals_by_image[0:3])

In [ ]:
aggs = { \
       'Total cells': 'sum', \
       'Spotted cells': 'sum', \
       'Spotted fraction': ['mean','sem']}
totals_by_well = totals_by_image.groupby(['Metadata_Plate', 'Metadata_Well']).agg(aggs).reset_index()
totals_by_well

In [ ]:
import numpy as np

df2 = totals_by_well

df2['row'] = df2['Metadata_Well'].str[0:1]
df2['col'] = df2['Metadata_Well'].str[1:4]

cols = [1,2,3,4,5,6,7,8,9,10,11,12]
rows = ['A', 'B', 'C','D','E','F','G','H'] 
plate = np.empty((len(rows),len(cols)))

measurement = []

for r in rows:
    for c in cols:
        #print(rows[r] + " " + cols[c])
        ir = rows.index(r)
        c_zfill = str(c).zfill(2)
        value = df2[(df2['row'] == r) & (df2['col'] == c_zfill)]['Spotted fraction','mean']
        if value.size != 0:
            plate[ir][c-1] = value
        else:
            plate[ir][c-1] = None
            
import plotly.graph_objects as go

fig = go.Figure(data=go.Heatmap(
                    x=cols,
                    y=rows,
                    z=plate,
                    hoverongaps = False))
fig.update_xaxes(tick0=1, dtick=1)
fig.update_yaxes(tick0=1, dtick=1)
fig.update_layout(xaxis_showgrid=False, yaxis_showgrid=False)
fig.update_yaxes(autorange="reversed")
fig.update_layout(title_text='Spotted fraction mean')

fig.show()

In [ ]:
import plotly.express as px

df2['sf'] = df2['Spotted fraction','mean']
df2['error'] = df2['Spotted fraction','sem']

# pick interesting wells
wells = ['C05','D05','F03']
df2 = df2[df2['Metadata_Well'].isin(wells)]

fig = px.bar(df2, y='sf', error_y='error', x='Metadata_Well', text='error', \
            labels={'sf':'Fraction of cells with spots','Metadata_Well':'Well'})
#fig.update_traces(texttemplate='%{text:.2s}', textposition='outside')
fig.update_layout(uniformtext_minsize=8, uniformtext_mode='hide')
fig.show()

In [ ]:
df2